In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup as BS

网页设置了AJAX的scrolling，一次得到的请求只能算是一页:
- 解决办法是查看network下的XHR，发现共有7个分页，这样可以用for loop把内容先放到一个变量wow中

结构很复杂：
- 首先，所有post都在<section id="page">下的<section class="content clearfix avatar-style-circle show-nav" id="posts">下的<div>
- 之后，尽管所有的posts的class都不一样，但共性是其下都有<section class="post">
- 在此之下，所有图片都在<figure class="post-content photoset with-caption">，这个就出现分歧了，只有上面的8/10
- 啊啊啊，越看越复杂越乱.. 今天能搞懂结构就不错了，先看第一个，妈的第一个还有reblog-link到另一个台湾网站
- 还有meta？？
- 从F12的source而不是elements更能得到所有图片信息, source的话小搜了下需要Selenium
- 这条也许有用：http://stackoverflow.com/questions/34546766/scraping-hidden-elements-using-beautifulsoup
- 每个<section class="post">的iframe最后的内容是一个隐形的#document，
- 关于#document，这个http://stackoverflow.com/questions/21474605/what-does-document-mean 也许有用
- 以及http://selenium-python.readthedocs.io/index.html 
- iframe下的文件！隐藏了图片但是貌似有结构，试试！
- 三种类型文件，普通iframe、一个图片的post、以及video

最后这条管用：
- 测试了f1_img，可以得到第一个iframe下的所有图片地址
- 测试每页几个iframe

其他需要考虑的要素：
- 有的是video，video如何爬，如何下载
- 标题在<figcaption>里

三大dict
- id_li or id_li_copy
- non_iframe_pics
- videos

In [23]:
# 得到所有的posts，解决AJAX scrolling
full_url = []
for i in range(1,8):
    full_url.append('http://hatsuneyuko.tumblr.com/page/%d' % i)
# 无法直接把内容添加到wow里，那么我们就做一个wow好了
wow = []
for i in range(7):
    wow.append(BS(requests.get(full_url[i]).content, 'html.parser'))
# 先解决第一页：wow[0] 正好10条post
wow[0].prettify


<bound method Tag.prettify of <!DOCTYPE html>
<script>var __pbpa = true;</script><script>var translated_warning_string = '\u8b66\u544a\uff1a\u9664\u975e\u6d4f\u89c8\u5668\u5730\u5740\u680f\u91cc\u663e\u793a\u7684\u662f  \u201chttps://www.tumblr.com/login\u201d\x0a\u5426\u5219\u6c38\u8fdc\u4e0d\u8981\u8f93\u5165\u4f60\u7684 Tumblr \u5bc6\u7801\u3002\x0a\x0a\u4f60\u5e94\u8be5\u80fd\u591f\u5728\u5730\u5740\u680f\u91cc\u770b\u5230\u7eff\u8272\u7684 \u201cTumblr, Inc.\u201d \u8bc6\u522b\u6807\u5fd7\u3002\x0a\x0a\u5783\u573e\u90ae\u4ef6\u53d1\u9001\u8005\u548c\u5176\u4ed6\u4e0d\u6cd5\u5206\u5b50\u4f1a\u4f7f\u7528\u4f2a\u9020\u7684\u8868\u5355\u6765\u5077\u53d6\u5bc6\u7801\u3002\x0a\x0aTumblr \u7edd\u4e0d\u4f1a\u8981\u6c42\u4f60\u4ece\u67d0\u4e2a\u7528\u6237\u7684\u535a\u5ba2\u4e0a\u767b\u5f55\u3002\x0a\x0a\u4f60\u771f\u7684\u975e\u5e38\u786e\u5b9a\u8981\u7ee7\u7eed\u5417\uff1f';</script><script language="javascript" src="http://assets.tumblr.com/assets/scripts/pre_tumblelog.js?_v=0bb0301f5c6

In [24]:
# 先试试 wow[0] 中的 iframe
wow[0].find_all('iframe')

[<iframe class="photoset" frameborder="0" height="-5" id="photoset_iframe_157741719791" name="photoset_iframe_157741719791" scrolling="no" src="http://hatsuneyuko.tumblr.com/post/157741719791/photoset_iframe/hatsuneyuko/tumblr_oa8ff5Pv1u1uuksk4/0/false" style="border:0px; background-color:transparent; overflow:hidden;" width="100%"></iframe>,
 <iframe allowtransparency="true" class="like_toggle" frameborder="0" height="24" id="like_iframe_157741719791" name="like_iframe_157741719791" scrolling="no" src="http://assets.tumblr.com/assets/html/like_iframe.html?_v=9c1e0782724156703eb4f7ba79dbe203#name=hatsuneyuko&amp;post_id=157741719791&amp;color=black&amp;rk=AyPXlyMD&amp;root_id=147320255733" width="24"></iframe>,
 <iframe class="photoset" frameborder="0" height="-1" id="photoset_iframe_157604626716" name="photoset_iframe_157604626716" scrolling="no" src="http://hatsuneyuko.tumblr.com/post/157604626716/photoset_iframe/hatsuneyuko/tumblr_oltk0r3GQO1ug4tgw/0/false" style="border:0px; backgr

In [30]:
len(wow[i].find_all('iframe'))

16

In [38]:
wow0_src_list = []
for i in wow[0].find_all('iframe'):
    if i.has_attr('src'):
        wow0_src_list.append(i.get('src'))
wow0_src_list

['http://hatsuneyuko.tumblr.com/post/157741719791/photoset_iframe/hatsuneyuko/tumblr_oa8ff5Pv1u1uuksk4/0/false',
 'http://assets.tumblr.com/assets/html/like_iframe.html?_v=9c1e0782724156703eb4f7ba79dbe203#name=hatsuneyuko&post_id=157741719791&color=black&rk=AyPXlyMD&root_id=147320255733',
 'http://hatsuneyuko.tumblr.com/post/157604626716/photoset_iframe/hatsuneyuko/tumblr_oltk0r3GQO1ug4tgw/0/false',
 'http://assets.tumblr.com/assets/html/like_iframe.html?_v=9c1e0782724156703eb4f7ba79dbe203#name=hatsuneyuko&post_id=157604626716&color=black&rk=NpQZiyhJ',
 'http://assets.tumblr.com/assets/html/like_iframe.html?_v=9c1e0782724156703eb4f7ba79dbe203#name=hatsuneyuko&post_id=156263160126&color=black&rk=6AxoaYHH',
 'http://hatsuneyuko.tumblr.com/post/155954659596/photoset_iframe/hatsuneyuko/tumblr_ofk713e7P51ugfvup/0/false',
 'http://assets.tumblr.com/assets/html/like_iframe.html?_v=9c1e0782724156703eb4f7ba79dbe203#name=hatsuneyuko&post_id=155954659596&color=black&rk=7DtwY5dX&root_id=1522516661

In [46]:
# get all the list yeah!!!
wow0_if_src = [s for s in wow0_src_list if 'false' in s]
wow0_if_src

['http://hatsuneyuko.tumblr.com/post/157741719791/photoset_iframe/hatsuneyuko/tumblr_oa8ff5Pv1u1uuksk4/0/false',
 'http://hatsuneyuko.tumblr.com/post/157604626716/photoset_iframe/hatsuneyuko/tumblr_oltk0r3GQO1ug4tgw/0/false',
 'http://hatsuneyuko.tumblr.com/post/155954659596/photoset_iframe/hatsuneyuko/tumblr_ofk713e7P51ugfvup/0/false',
 'http://hatsuneyuko.tumblr.com/post/155718251666/photoset_iframe/hatsuneyuko/tumblr_ojmcvuO4K01ug4tgw/0/false',
 'http://hatsuneyuko.tumblr.com/post/155486101401/photoset_iframe/hatsuneyuko/tumblr_ojd9lyZl4D1ug4tgw/0/false',
 'http://hatsuneyuko.tumblr.com/post/155064827576/photoset_iframe/hatsuneyuko/tumblr_oiw2qliD2d1ug4tgw/0/false',
 'http://hatsuneyuko.tumblr.com/post/155025269636/photoset_iframe/hatsuneyuko/tumblr_oiuo6mvZCy1ug4tgw/0/false']

### 试试第一个iframe

In [5]:
response = requests.get('http://hatsuneyuko.tumblr.com/post/157741719791/photoset_iframe/hatsuneyuko/tumblr_oa8ff5Pv1u1uuksk4/0/false')
f1 = BS(response.content, 'html.parser')
f1.prettify

<bound method Tag.prettify of 
<!DOCTYPE HTML>
<html><head><meta content="noindex,nofollow" name="robots"/>
<style type="text/css">
        body {
            background: transparent;
            padding: 0;
            margin: 0;
            overflow: hidden;
        }

        .shadow {
            padding: 5px 10px;
        }

        img, a img {
            border-width: 0;
            -webkit-tap-highlight-color: rgba(0,0,0,0);
        }

        .shadow a {
            -webkit-box-shadow:  0px 0px 3px 1px rgba(0, 0, 0, 0.4);
            box-shadow:  0px 0px 3px 1px rgba(0, 0, 0, 0.4);
        }

    .photoset {
        -moz-transform-origin: 0 0;
    }

    .photoset .photoset_row {
        width: 700px;
        white-space: nowrap;
        overflow: hidden;
        margin-top: 10px;
    }

    .photoset.shadow .photoset_row {
        overflow: visible;
    }

    .photoset.mobile .photoset_row {
        margin-top: 17px;
    }

    .photoset .photoset_row:first-child {
        

In [18]:
img_list = f1.find_all('img')


[<img alt="" class="" data-pin-description="哒哒耻嘟" data-pin-url="http://nvxing.tumblr.com/post/147320255733" src="http://68.media.tumblr.com/223090bed2fc9d5ad6d9caa7150fcea5/tumblr_oa8ff5Pv1u1uuksk4o3_540.jpg" style="
                                         width:700px;"/>,
 <img alt="" class="" data-pin-description="哒哒耻嘟" data-pin-url="http://nvxing.tumblr.com/post/147320255733" src="http://68.media.tumblr.com/8c083ef63379af6c0363b5a8ea62b6b5/tumblr_oa8ff5Pv1u1uuksk4o1_400.jpg" style="
                                         width:345px;"/>,
 <img alt="" class="" data-pin-description="哒哒耻嘟" data-pin-url="http://nvxing.tumblr.com/post/147320255733" src="http://68.media.tumblr.com/919de13855e76b4eac0aa7d3f8906cfc/tumblr_oa8ff5Pv1u1uuksk4o9_400.jpg" style="
                                         width:345px;"/>,
 <img alt="" class="" data-pin-description="哒哒耻嘟" data-pin-url="http://nvxing.tumblr.com/post/147320255733" src="http://68.media.tumblr.com/b702a7087b22e3620a675b02864c0695/tu

In [19]:
len(img_list)

9

In [21]:
f1_img = []
for i in img_list:
    f1_img.append(i.get('src'))

In [22]:
f1_img

['http://68.media.tumblr.com/223090bed2fc9d5ad6d9caa7150fcea5/tumblr_oa8ff5Pv1u1uuksk4o3_540.jpg',
 'http://68.media.tumblr.com/8c083ef63379af6c0363b5a8ea62b6b5/tumblr_oa8ff5Pv1u1uuksk4o1_400.jpg',
 'http://68.media.tumblr.com/919de13855e76b4eac0aa7d3f8906cfc/tumblr_oa8ff5Pv1u1uuksk4o9_400.jpg',
 'http://68.media.tumblr.com/b702a7087b22e3620a675b02864c0695/tumblr_oa8ff5Pv1u1uuksk4o8_540.jpg',
 'http://68.media.tumblr.com/e7ccce0fcef5f488fff84e79a66a38e9/tumblr_oa8ff5Pv1u1uuksk4o5_540.jpg',
 'http://68.media.tumblr.com/2138c57c70bc976fbcc7b8e4be18f353/tumblr_oa8ff5Pv1u1uuksk4o7_540.jpg',
 'http://68.media.tumblr.com/9bb2056771406e35e10eb3acf30f8a5e/tumblr_oa8ff5Pv1u1uuksk4o2_400.jpg',
 'http://68.media.tumblr.com/24ed39a9033449f89e4b02e8b1751733/tumblr_oa8ff5Pv1u1uuksk4o6_400.jpg',
 'http://68.media.tumblr.com/9451cf7c42ad0450001e69646fedb1ac/tumblr_oa8ff5Pv1u1uuksk4o4_540.jpg']

## 得到所有img的多个list

- 需要所有page的full_url, 此为第一个大list
- 每个page下找到正确的if_src，此为第二个大list
- 每个if_src下，找到所有图片，此为第三个大list

In [66]:
# use dict to create a dict of pages, each has an empty list
page_iframe = {}
for i in range(7):
    page_iframe['page%d' % (i+1)] = []
page_iframe

{'page1': [],
 'page2': [],
 'page3': [],
 'page4': [],
 'page5': [],
 'page6': [],
 'page7': []}

In [67]:
# put each page's if_scr to the list
for i,k in enumerate(wow):
    for j in k.find_all('iframe'):
        if j.has_attr('src'):
            page_iframe['page%d' % (i+1)].append(j.get('src'))
    page_iframe['page%d' % (i+1)] = [s for s in page_iframe['page%d' % (i+1)] if 'false' in s]
page_iframe

{'page1': ['http://hatsuneyuko.tumblr.com/post/157741719791/photoset_iframe/hatsuneyuko/tumblr_oa8ff5Pv1u1uuksk4/0/false',
  'http://hatsuneyuko.tumblr.com/post/157604626716/photoset_iframe/hatsuneyuko/tumblr_oltk0r3GQO1ug4tgw/0/false',
  'http://hatsuneyuko.tumblr.com/post/155954659596/photoset_iframe/hatsuneyuko/tumblr_ofk713e7P51ugfvup/0/false',
  'http://hatsuneyuko.tumblr.com/post/155718251666/photoset_iframe/hatsuneyuko/tumblr_ojmcvuO4K01ug4tgw/0/false',
  'http://hatsuneyuko.tumblr.com/post/155486101401/photoset_iframe/hatsuneyuko/tumblr_ojd9lyZl4D1ug4tgw/0/false',
  'http://hatsuneyuko.tumblr.com/post/155064827576/photoset_iframe/hatsuneyuko/tumblr_oiw2qliD2d1ug4tgw/0/false',
  'http://hatsuneyuko.tumblr.com/post/155025269636/photoset_iframe/hatsuneyuko/tumblr_oiuo6mvZCy1ug4tgw/0/false'],
 'page2': ['http://hatsuneyuko.tumblr.com/post/154963484926/photoset_iframe/hatsuneyuko/tumblr_oirynlIxft1ug4tgw/0/false',
  'http://hatsuneyuko.tumblr.com/post/154888968611/photoset_iframe/ha

In [70]:
# post id
'http://hatsuneyuko.tumblr.com/post/157741719791/photoset_iframe/hatsuneyuko/tumblr_oa8ff5Pv1u1uuksk4/0/false'.split('/')[4]

'157741719791'

In [81]:
# key=post_id : value=imgs
print([s for s in page_iframe['page1'] if '155486101401' in s])

['http://hatsuneyuko.tumblr.com/post/155486101401/photoset_iframe/hatsuneyuko/tumblr_ojd9lyZl4D1ug4tgw/0/false']


In [91]:
# 原来的 dict 太复杂，先改成post_id : src
id_li = {}
for i in page_iframe:
    for j in page_iframe[i]:
        id_li[j.split('/')[4]] = j
id_li

{'146547040456': 'http://hatsuneyuko.tumblr.com/post/146547040456/photoset_iframe/hatsuneyuko/tumblr_o9fd4fOg4g1ug4tgw/0/false',
 '146547299351': 'http://hatsuneyuko.tumblr.com/post/146547299351/photoset_iframe/hatsuneyuko/tumblr_o9fdo8b4w21ug4tgw/0/false',
 '146593902096': 'http://hatsuneyuko.tumblr.com/post/146593902096/photoset_iframe/hatsuneyuko/tumblr_o9gzj0Xdsu1ug4tgw/0/false',
 '146601431436': 'http://hatsuneyuko.tumblr.com/post/146601431436/photoset_iframe/hatsuneyuko/tumblr_o9heizr5nV1ug4tgw/0/false',
 '146605379546': 'http://hatsuneyuko.tumblr.com/post/146605379546/photoset_iframe/hatsuneyuko/tumblr_o9hkrdsSRx1ug4tgw/0/false',
 '146835645971': 'http://hatsuneyuko.tumblr.com/post/146835645971/photoset_iframe/hatsuneyuko/tumblr_o9q812EhSI1ug4tgw/0/false',
 '146886473276': 'http://hatsuneyuko.tumblr.com/post/146886473276/photoset_iframe/hatsuneyuko/tumblr_o9s4ox9ix11ug4tgw/0/false',
 '147544362031': 'http://hatsuneyuko.tumblr.com/post/147544362031/photoset_iframe/hatsuneyuko/tum

In [107]:
for i in id_li:
    soup = BS(requests.get(id_li[i]).content, 'html.parser').find_all('img')
    soup = [s.get('src') for s in soup]
    id_li[i] = soup
id_li

{'146547040456': ['http://68.media.tumblr.com/c5cd9ffb752b2b7f0d6e923ef828b052/tumblr_o9fd4fOg4g1ug4tgwo1_540.jpg',
  'http://68.media.tumblr.com/43770f60fc269ed381cd655012cc77e1/tumblr_o9fd4fOg4g1ug4tgwo2_540.jpg',
  'http://68.media.tumblr.com/70f03e86f88e778259b38eea9d51755d/tumblr_o9fd4fOg4g1ug4tgwo3_540.jpg',
  'http://68.media.tumblr.com/252d2e7f7188a015a3e4e8d020fd76c8/tumblr_o9fd4fOg4g1ug4tgwo8_540.jpg',
  'http://68.media.tumblr.com/ab4e4f4eaf9a65dfffcf52835e0fd15b/tumblr_o9fd4fOg4g1ug4tgwo6_540.jpg',
  'http://68.media.tumblr.com/8eafca88fb456733d580b4687a4667ed/tumblr_o9fd4fOg4g1ug4tgwo9_540.jpg',
  'http://68.media.tumblr.com/a19fcd3c391bcf7ff04ca5858802f155/tumblr_o9fd4fOg4g1ug4tgwo4_540.jpg',
  'http://68.media.tumblr.com/a7a4729cb357eb8148d1a7cd9e4f7752/tumblr_o9fd4fOg4g1ug4tgwo5_540.jpg',
  'http://68.media.tumblr.com/970ec058d4283b0d37e39d626bb1a510/tumblr_o9fd4fOg4g1ug4tgwo7_540.jpg'],
 '146547299351': ['http://68.media.tumblr.com/30470c536d065f23aac440195b0e84c7/tumb

In [126]:
id_li_copy = id_li.copy() # make a copy in case sth went wrong

In [111]:
# create directories
import os

In [130]:
pic_path = 'D:\\Google Drive\\Python_Exercise\\Spider_1st_Youmu\\pictures\\'

- pic_path是base path，可以

In [249]:
# this is magic
for a in id_li_copy:
    os.makedirs(pic_path + a) # 根据 post_id 创建文件夹
    # 解析每个 post_id 的网址，下载到对应文件夹
    for b, i in enumerate(id_li_copy[a]): # b=index
        img = requests.get(i).content
        with open(pic_path + a + '\\%s.jpg' % (b+1), 'wb') as handler:
            handler.write(img)

### non_iframe_pics

- 不带iframe的post的特点是只有一张图片，可以直接找img里带height的
- 新建一个dict来处理这些pic，key=图片的alt

In [152]:
# 找到所有的这些post，让入一个list
blah = []
for i in range(7):
    # 如果直接append blah的话，blah会是一个包含str和list的list
    l = [s for s in wow[i].find_all('img') if s.has_attr('height')]
    for j in l:
        blah.append(j) 
blah

[<img alt="See you next year." height="769" src="http://68.media.tumblr.com/0a1fa3a151ed9c699739cadfeb936dc4/tumblr_ok8ihcm5sP1ug4tgwo1_1280.jpg" width="1280"/>,
 <img alt="Theatre." height="816" src="http://68.media.tumblr.com/a6d72caf6c291c6a17d5b77fdce71d7c/tumblr_oiy9kma1Yw1ug4tgwo1_1280.jpg" width="1204"/>,
 <img alt="高铁露出w" height="724" src="http://68.media.tumblr.com/36167931b30e5f0806a247f2fe1fc1ff/tumblr_oi6j4wsyLD1ug4tgwo1_1280.jpg" width="1280"/>,
 <img alt="" height="979" src="http://68.media.tumblr.com/0fbf14924a2a9b02dd4ffd6ddd470728/tumblr_ohihmt6uqj1ug4tgwo1_1280.gif" width="700"/>,
 <img alt="手比爱心" height="1280" src="http://68.media.tumblr.com/b87b866ed527d2e9a129e76d236f745c/tumblr_oherzaULWA1ug4tgwo1_1280.jpg" width="960"/>,
 <img alt="男厕露出~" height="1280" src="http://68.media.tumblr.com/92fdeb8026f06e739f8e0a4e734bae10/tumblr_oewb8cLaOp1ug4tgwo1_1280.jpg" width="858"/>,
 <img alt="Family Mart." height="773" src="http://68.media.tumblr.com/eeccb733a674b27797cbafb7ae5

In [155]:
non_iframe_pics = {}
for i in blah:
    non_iframe_pics[i.get('alt')] = i.get('src')
non_iframe_pics
# 注意里面有一个是 gif

{'': 'http://68.media.tumblr.com/0fbf14924a2a9b02dd4ffd6ddd470728/tumblr_ohihmt6uqj1ug4tgwo1_1280.gif',
 '1933.': 'http://68.media.tumblr.com/503fb924b2f00fecd7c02dd5d9b4eea1/tumblr_o9fxfizDU61ug4tgwo1_1280.jpg',
 'Always.': 'http://68.media.tumblr.com/8171f8da0a6cf48a1baa6124e8823cee/tumblr_oa7tn9njSY1ug4tgwo1_1280.jpg',
 'Family Mart.': 'http://68.media.tumblr.com/eeccb733a674b27797cbafb7ae5a03fd/tumblr_oao6gdfoXX1ug4tgwo1_1280.jpg',
 'Here she comes.': 'http://68.media.tumblr.com/307f694b57afffd6d36502d082fdfed0/tumblr_oab80nNtca1ug4tgwo1_1280.jpg',
 'See you next year.': 'http://68.media.tumblr.com/0a1fa3a151ed9c699739cadfeb936dc4/tumblr_ok8ihcm5sP1ug4tgwo1_1280.jpg',
 'Silence.': 'http://68.media.tumblr.com/b0445fae493dbe187f74b514d608acb1/tumblr_o9mzq8A3af1ug4tgwo1_1280.jpg',
 'The trailer.': 'http://68.media.tumblr.com/45661dff8847854a76a5a3cc1b8a8233/tumblr_o9gzt7Xl7h1ug4tgwo1_1280.jpg',
 'Theatre.': 'http://68.media.tumblr.com/a6d72caf6c291c6a17d5b77fdce71d7c/tumblr_oiy9kma1Yw

### Videos

- 首先定位iframe其中有video的, 共性 {'class':'embed_iframe tumblr_video_iframe'}
- 接着进入此url，找到<video>

In [206]:
video_list = []
for i in range(7):
    temp = wow[i].find_all('iframe', {'class':'embed_iframe tumblr_video_iframe'})
    for j in temp:
        video_list.append(j)
video_list = [s.get('src') for s in video_list]
video_list

['https://www.tumblr.com/video/hatsuneyuko/155077721581/700/',
 'https://www.tumblr.com/video/hatsuneyuko/154150707113/700/',
 'https://www.tumblr.com/video/hatsuneyuko/150021529076/700/']

In [231]:
# 找到video
video_list_deep = []
for i in range(len(video_list)):
    soup = BS(requests.get(video_list[i]).content, 'html.parser').find_all('video')
    for j in soup:
        video_list_deep.append(j)
for i in range(len(video_list_deep)):
    video_list_deep[i] = video_list_deep[i].find('source').get('src')
video_list_deep

['https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/155077721581/tumblr_o4xhru4f951ugfvup',
 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/154150707113/tumblr_ohsv2daC2R1ug4tgw/480',
 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/150021529076/tumblr_od2oortMro1ug4tgw/480']

In [234]:
# a dict
videos = {}
for i in range(len(video_list_deep)):
    videos[video_list_deep[i].split('/')[5]] = video_list_deep[i]
videos

{'150021529076': 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/150021529076/tumblr_od2oortMro1ug4tgw/480',
 '154150707113': 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/154150707113/tumblr_ohsv2daC2R1ug4tgw/480',
 '155077721581': 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/155077721581/tumblr_o4xhru4f951ugfvup'}

## 集合二个dict，和video

三个dict为: id_list_copy, non_iframe_pics, videos

In [250]:
# make 3 copies
a = id_li_copy.copy()
b = non_iframe_pics.copy()
c = videos.copy()

In [246]:
# b.update(a)
# 现在b包含了所有的a元素

下载non_iframe_pics

In [253]:
# download b 错误，不知为何
for i in b:
    os.makedirs(pic_path + i) # 根据 post_id 创建文件夹
    # 解析每个 post_id 的网址，下载到对应文件夹
    for d, j in enumerate(b[i]): # b=index
        img = requests.get(j).content
        with open(pic_path + i + '\\%s.jpg' % (d+1), 'wb') as handler:
            handler.write(img)

MissingSchema: Invalid URL 'h': No schema supplied. Perhaps you meant http://h?

In [251]:
b

{'': 'http://68.media.tumblr.com/0fbf14924a2a9b02dd4ffd6ddd470728/tumblr_ohihmt6uqj1ug4tgwo1_1280.gif',
 '1933.': 'http://68.media.tumblr.com/503fb924b2f00fecd7c02dd5d9b4eea1/tumblr_o9fxfizDU61ug4tgwo1_1280.jpg',
 'Always.': 'http://68.media.tumblr.com/8171f8da0a6cf48a1baa6124e8823cee/tumblr_oa7tn9njSY1ug4tgwo1_1280.jpg',
 'Family Mart.': 'http://68.media.tumblr.com/eeccb733a674b27797cbafb7ae5a03fd/tumblr_oao6gdfoXX1ug4tgwo1_1280.jpg',
 'Here she comes.': 'http://68.media.tumblr.com/307f694b57afffd6d36502d082fdfed0/tumblr_oab80nNtca1ug4tgwo1_1280.jpg',
 'See you next year.': 'http://68.media.tumblr.com/0a1fa3a151ed9c699739cadfeb936dc4/tumblr_ok8ihcm5sP1ug4tgwo1_1280.jpg',
 'Silence.': 'http://68.media.tumblr.com/b0445fae493dbe187f74b514d608acb1/tumblr_o9mzq8A3af1ug4tgwo1_1280.jpg',
 'The trailer.': 'http://68.media.tumblr.com/45661dff8847854a76a5a3cc1b8a8233/tumblr_o9gzt7Xl7h1ug4tgwo1_1280.jpg',
 'Theatre.': 'http://68.media.tumblr.com/a6d72caf6c291c6a17d5b77fdce71d7c/tumblr_oiy9kma1Yw

In [259]:
# 清理下名字
b['gif'] = b.pop('')

KeyError: ''

In [260]:
#继续清理
for i in b:
    if i[-1] == '.' or i[-1] == '。':
        b[i[:-1]] = b.pop(i)

In [278]:
b
# 还剩1933.

{'1933': 'http://68.media.tumblr.com/503fb924b2f00fecd7c02dd5d9b4eea1/tumblr_o9fxfizDU61ug4tgwo1_1280.jpg',
 'Always': 'http://68.media.tumblr.com/8171f8da0a6cf48a1baa6124e8823cee/tumblr_oa7tn9njSY1ug4tgwo1_1280.jpg',
 'Family Mart': 'http://68.media.tumblr.com/eeccb733a674b27797cbafb7ae5a03fd/tumblr_oao6gdfoXX1ug4tgwo1_1280.jpg',
 'Here she comes': 'http://68.media.tumblr.com/307f694b57afffd6d36502d082fdfed0/tumblr_oab80nNtca1ug4tgwo1_1280.jpg',
 'See you next year': 'http://68.media.tumblr.com/0a1fa3a151ed9c699739cadfeb936dc4/tumblr_ok8ihcm5sP1ug4tgwo1_1280.jpg',
 'Silence': 'http://68.media.tumblr.com/b0445fae493dbe187f74b514d608acb1/tumblr_o9mzq8A3af1ug4tgwo1_1280.jpg',
 'The trailer': 'http://68.media.tumblr.com/45661dff8847854a76a5a3cc1b8a8233/tumblr_o9gzt7Xl7h1ug4tgwo1_1280.jpg',
 'Theatre': 'http://68.media.tumblr.com/a6d72caf6c291c6a17d5b77fdce71d7c/tumblr_oiy9kma1Yw1ug4tgwo1_1280.jpg',
 'bloomer girl': 'http://68.media.tumblr.com/f1dc044bf314ba836aff48a229008f7c/tumblr_o9mzo9

In [262]:
b['1933'] = b.pop('1933.')

In [273]:
b['The trailer'] = b.pop('The trailer.')

KeyError: 'The trailer.'

In [274]:
b['bloomer girl'] = b.pop('ブルマー少女')

In [284]:
#test 不止为何出现错误，只能取消了enumerate
os.makedirs(pic_path +  'bloomer girl')
img = requests.get(b[ 'bloomer girl']).content
with open(pic_path +  'bloomer girl' + '\\%s.gif' % 1, 'wb') as handler:
    handler.write(img)

In [285]:
# download b
for i in b:
    os.makedirs(pic_path + i)
    img = requests.get(b[i]).content
    with open(pic_path + i + '\\%s.jpg' % 1, 'wb') as  handler:
            handler.write(img)

下载videos

In [289]:
videos

{'150021529076': 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/150021529076/tumblr_od2oortMro1ug4tgw/480',
 '154150707113': 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/154150707113/tumblr_ohsv2daC2R1ug4tgw/480',
 '155077721581': 'https://hatsuneyuko.tumblr.com/video_file/t:qSc2wep8RMsm-JQ3PbviUQ/155077721581/tumblr_o4xhru4f951ugfvup'}

In [291]:
video_path = 'D:\\Google Drive\\Python_Exercise\\Spider_1st_Youmu\\videos\\'

In [292]:
for i in videos:
    os.makedirs(video_path + i)
    mp4 = requests.get(videos[i]).content
    with open(video_path + i + '\\%s.mp4' % 1, 'wb') as  handler:
            handler.write(mp4)